In [ ]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from tb_incubator.constants import set_project_base_path, QUANTILES, image_path, MODEL_TIMES
from tb_incubator.calibrate import plot_output_ranges, get_quantile_outputs
from tb_incubator.plotting import get_combined_plot
from tb_incubator.input import load_targets
from estival.sampling import tools as esamp
import plotly.graph_objects as go

project_paths = set_project_base_path("../tb_incubator/")
calib_out = project_paths["OUT_PATH"]

In [ ]:
targets = load_targets()
out_req = ["notification_log", "adults_prevalence_pulmonary_log"]

file_suffixes = ["xpert_only_25000d10000t_01", "no_xpert_no_cdr_25000d10000t_01"]
quantile_outputs = get_quantile_outputs(file_suffixes, calib_out)

In [ ]:
epi_plots = {}
indicators = ['notification', 'adults_prevalence_pulmonary', 'incidence', 'mortality']

for indicator in indicators:
    epi_plots[indicator] = plot_output_ranges(
        quantile_outputs["xpert_only_25000d10000t_01"], targets, 
        [f'{indicator}'], 1, 2015, 2035, 2013, show_legend=False, show_title=False
    )

In [ ]:
epi_aspects = plot_output_ranges(quantile_outputs["xpert_only_25000d10000t_01"], targets, 
                                 ['adults_prevalence_pulmonary', 'incidence', 'mortality', 'percentage_latent'], 
                                 2, 2010, 2025, 2040, show_title=False,show_legend=False)
#epi_aspects.write_image(image_path / f'tb_epi_{file_suffixes[0]}.png', scale=3)
epi_aspects

In [ ]:
epi_aspects_hist = plot_output_ranges(quantile_outputs["xpert_only_25000d10000t_01"], targets, 
                                      ['adults_prevalence_pulmonary', 'incidence', 'mortality', 'percentage_latent'], 2, MODEL_TIMES[0], MODEL_TIMES[1], 2040, show_title=False,show_legend=False, history=True, show_target_data=False)
#epi_aspects_hist.write_image(image_path / f'tb_epi_hist_{file_suffixes[0]}.png', width=800, height=600,scale=3)

In [ ]:
detect = plot_output_ranges(quantile_outputs["xpert_only_25000d10000t_01"], targets, 
                            ['base_detection', 'diagnostic_capacity', 'final_detection', 'notif_ratio'], 2, 2010, 2025, 2040, show_title=False,show_legend=False)
#detect.write_image(image_path / f'tb_detect_{file_suffixes[0]}.png', width=1400, height=500,scale=3)

In [ ]:
tot_pop = plot_output_ranges(quantile_outputs["xpert_only_25000d10000t_01"], targets, 
                             ['total_population'], 1, 2010, 2025, 2040, show_legend=True, show_title=False, legend_name="Population estimates")
tot_pop.add_trace(
    go.Scatter(
        x=targets["census_pop"].index,
        y=targets["census_pop"],
        mode="markers",
        marker=dict(
            symbol="circle",
            size=4,
            color="black",
            line=dict(width=1)  
        ), 
        name="Census data",
    )
)

tot_pop.update_layout(
    height=300, 
    width=550,
    showlegend=False,
    margin=dict(
        l=50,   # left margin
        r=50,   # right margin
        t=30,   # top margin
        b=30    # bottom margin
    )
)
#tot_pop.write_image(image_path / f'total_pop_comp_{file_suffixes[0]}.png', scale=3)

In [ ]:
import plotly.graph_objects as go

detect_plot = plot_output_ranges(quantile_outputs["xpert_only_25000d10000t_01"], targets, ['passive_notification_raw'], 1, 2010, 2025, 2040, show_legend=True, show_title=False, colour="255, 127, 0")
notif_plot = plot_output_ranges(quantile_outputs["xpert_only_25000d10000t_01"], targets, ['notification'], 1, 2010, 2025, 2040, show_legend=False, show_title=False, show_target_data=False)
combined_fig = go.Figure()

for trace in detect_plot.data:
    combined_fig.add_trace(trace)

for trace in notif_plot.data:
    combined_fig.add_trace(trace)

combined_fig.update_layout(
    xaxis_title="Year",
    yaxis_title="Value",
    legend_title="Indicator",
    height=600,
    showlegend=True
)

combined_fig.show()
#combined_fig.write_image(image_path / f'tb_detect_vs_notif_{file_suffixes[0]}.png', width=1400, height=500,scale=3)